# Day 3 - Lab 1: AI-Driven Backend Development (Solution)

**Objective:** Generate a complete FastAPI backend application, including Pydantic and SQLAlchemy models, and then perform the critical engineering task of integrating the generated code with the live SQLite database created on Day 2.

**Introduction:**
This solution notebook provides the complete code and prompts for generating and assembling the database-connected API. It highlights the workflow of generating components separately and then integrating them, a common pattern in AI-assisted development.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

**Explanation:**
We load our `schema.sql` artifact, which will be the primary context for our code generation prompts. Having the database schema is essential for the LLM to accurately generate models (both Pydantic and SQLAlchemy) and endpoints that match our data structure.

In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output, recommended_models_table

client, model_name, api_provider = setup_llm_client(model_name="meta-llama/Llama-4-Scout-17B-16E-Instruct")

# Load the SQL schema from Day 2
sql_schema = load_artifact("artifacts/schema.sql")
if not sql_schema:
    print("Warning: Could not load schema.sql. Lab may not function correctly.")

✅ LLM Client configured: Using 'huggingface' with model 'meta-llama/Llama-4-Scout-17B-16E-Instruct'


/Users/armando/Documents/Github/AG-AISOFTDEV/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
recommended_models_table()

| Model | Provider | Vision | Image Gen | Audio Transcription | Context Window | Max Output Tokens |
|---|---|---|---|---|---|---|
| claude-opus-4-1-20250805 | anthropic | ✅ | ❌ | ❌ | 200,000 | 100,000 |
| claude-opus-4-20250514 | anthropic | ✅ | ❌ | ❌ | 200,000 | 100,000 |
| claude-sonnet-4-20250514 | anthropic | ✅ | ❌ | ❌ | 1,000,000 | 100,000 |
| codex-mini | openai | ✅ | ❌ | ❌ | 200,000 | 100,000 |
| dall-e-3 | openai | ❌ | ✅ | ❌ | - | - |
| deepseek-ai/DeepSeek-V3 | huggingface | ❌ | ❌ | ❌ | 128,000 | 100,000 |
| deepseek-ai/DeepSeek-V3-Small | huggingface | ❌ | ❌ | ❌ | 128,000 | 100,000 |
| deepseek-ai/DeepSeek-VL2 | huggingface | ✅ | ❌ | ❌ | 32,000 | 8,000 |
| deepseek-ai/DeepSeek-VL2-Small | huggingface | ✅ | ❌ | ❌ | 32,000 | 8,000 |
| deepseek-ai/DeepSeek-VL2-Tiny | huggingface | ✅ | ❌ | ❌ | 32,000 | 8,000 |
| deepseek-ai/Janus-Pro-7B | huggingface | ✅ | ❌ | ❌ | 8,192 | 2,048 |
| gemini-2.0-flash | google | ✅ | ❌ | ❌ | 1,048,576 | 8,192 |
| gemini-2.0-flash-lite | google | ✅ | ❌ | ❌ | 1,048,576 | 8,192 |
| gemini-2.0-flash-live-001 | google | ✅ | ❌ | ❌ | 1,048,576 | 8,192 |
| gemini-2.5-flash | google | ✅ | ❌ | ❌ | 1,048,576 | 65,536 |
| gemini-2.5-flash-image-preview | google | ✅ | ✅ | ❌ | 32,768 | 32,768 |
| gemini-2.5-flash-lite | google | ✅ | ❌ | ❌ | 1,048,576 | 65,536 |
| gemini-2.5-pro | google | ✅ | ❌ | ❌ | 1,048,576 | 65,536 |
| gemini-deep-think | google | ✅ | ❌ | ❌ | 1,000,000 | 100,000 |
| gemini-live-2.5-flash-preview | google | ✅ | ❌ | ❌ | 1,048,576 | 8,192 |
| gemini-veo-3 | google | ✅ | ❌ | ❌ | - | - |
| google-cloud/speech-to-text/latest_long | google | ❌ | ❌ | ✅ | - | - |
| google-cloud/speech-to-text/latest_short | google | ❌ | ❌ | ✅ | - | - |
| gpt-4.1 | openai | ✅ | ❌ | ❌ | 1,000,000 | 32,000 |
| gpt-4.1-mini | openai | ✅ | ❌ | ❌ | 1,000,000 | 32,000 |
| gpt-4.1-nano | openai | ✅ | ❌ | ❌ | 1,000,000 | 32,000 |
| gpt-4.5 | openai | ✅ | ❌ | ❌ | 128,000 | 16,384 |
| gpt-4o | openai | ✅ | ❌ | ❌ | 128,000 | 16,384 |
| gpt-4o-mini | openai | ✅ | ❌ | ❌ | 128,000 | 16,384 |
| gpt-5-2025-08-07 | openai | ✅ | ❌ | ❌ | 400,000 | 128,000 |
| gpt-5-mini-2025-08-07 | openai | ✅ | ❌ | ❌ | 400,000 | 128,000 |
| gpt-5-nano-2025-08-07 | openai | ✅ | ❌ | ❌ | 400,000 | 128,000 |
| gpt-image-1 | openai | ✅ | ✅ | ❌ | - | - |
| imagen-3.0-generate-002 | google | ❌ | ✅ | ❌ | - | - |
| imagen-4.0-generate-001 | google | ❌ | ✅ | ❌ | 480 | - |
| meta-llama/Llama-3.3-70B-Instruct | huggingface | ❌ | ❌ | ❌ | 4,096 | 1,024 |
| meta-llama/Llama-4-Maverick-17B-128E-Instruct | huggingface | ✅ | ❌ | ❌ | 1,000,000 | 100,000 |
| meta-llama/Llama-4-Scout-17B-16E-Instruct | huggingface | ✅ | ❌ | ❌ | 10,000,000 | 100,000 |
| mistralai/Mistral-7B-Instruct-v0.3 | huggingface | ❌ | ❌ | ❌ | 32,768 | 8,192 |
| o3 | openai | ✅ | ❌ | ❌ | 200,000 | 100,000 |
| o4-mini | openai | ✅ | ❌ | ❌ | 200,000 | 100,000 |
| tokyotech-llm/Llama-3.1-Swallow-70B-Instruct-v0.3 | huggingface | ❌ | ❌ | ❌ | 4,096 | 1,024 |
| tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.5 | huggingface | ❌ | ❌ | ❌ | 4,096 | 1,024 |
| whisper-1 | openai | ❌ | ❌ | ✅ | - | - |

'| Model | Provider | Vision | Image Gen | Audio Transcription | Context Window | Max Output Tokens |\n|---|---|---|---|---|---|---|\n| claude-opus-4-1-20250805 | anthropic | ✅ | ❌ | ❌ | 200,000 | 100,000 |\n| claude-opus-4-20250514 | anthropic | ✅ | ❌ | ❌ | 200,000 | 100,000 |\n| claude-sonnet-4-20250514 | anthropic | ✅ | ❌ | ❌ | 1,000,000 | 100,000 |\n| codex-mini | openai | ✅ | ❌ | ❌ | 200,000 | 100,000 |\n| dall-e-3 | openai | ❌ | ✅ | ❌ | - | - |\n| deepseek-ai/DeepSeek-V3 | huggingface | ❌ | ❌ | ❌ | 128,000 | 100,000 |\n| deepseek-ai/DeepSeek-V3-Small | huggingface | ❌ | ❌ | ❌ | 128,000 | 100,000 |\n| deepseek-ai/DeepSeek-VL2 | huggingface | ✅ | ❌ | ❌ | 32,000 | 8,000 |\n| deepseek-ai/DeepSeek-VL2-Small | huggingface | ✅ | ❌ | ❌ | 32,000 | 8,000 |\n| deepseek-ai/DeepSeek-VL2-Tiny | huggingface | ✅ | ❌ | ❌ | 32,000 | 8,000 |\n| deepseek-ai/Janus-Pro-7B | huggingface | ✅ | ❌ | ❌ | 8,192 | 2,048 |\n| gemini-2.0-flash | google | ✅ | ❌ | ❌ | 1,048,576 | 8,192 |\n| gemini-2.0-flash-lite

## Step 2: The Challenges - Solutions

### Challenge 1 (Foundational): Generating Code with In-Memory Logic

**Explanation:**
This prompt generates a fully functional but simplified version of our application. By asking for an in-memory database, we allow the LLM to focus on generating the correct API structure, endpoints, and Pydantic models without the added complexity of database integration code. This gives us a clean, working baseline that we can build upon.

In [3]:
in_memory_api_prompt = f"""
You are a senior Python developer creating a FastAPI application for a new hire onboarding tool.

Based on the following SQL schema, generate a single Python script for a `main.py` file that includes:
1.  All necessary FastAPI imports.
2.  Pydantic models for creating and reading `User` resources. Include fields for `id`, `name`, `email`, and `role`.
3.  A simple in-memory list to act as a fake database for users.
4.  Complete FastAPI CRUD endpoints for the `/users` path (POST, GET all, GET by ID).
5.  The endpoints should perform their logic on the in-memory list.

**SQL Schema Context:**
```sql
{sql_schema}
```

Output only the raw Python code.
"""

print("--- Generating FastAPI app with in-memory database ---")
if sql_schema:
    generated_api_code = get_completion(in_memory_api_prompt, client, model_name, api_provider)
    cleaned_code = clean_llm_output(generated_api_code, language='python')
    # Save this code to a temporary reference file
    save_artifact(cleaned_code, "app/main_in_memory.py")
    print("Saved in-memory API to app/main_in_memory.py")
else:
    print("Skipping API generation because schema is missing.")

--- Generating FastAPI app with in-memory database ---
✅ Successfully saved artifact to: app/main_in_memory.py
Saved in-memory API to app/main_in_memory.py


### Challenge 2 (Intermediate): Generating Database Models and Session Code

**Explanation:**
This prompt is highly specific. It asks for the two key components needed for database connectivity in a modern Python application: the ORM (Object-Relational Mapping) models and the session management code. 
-   **SQLAlchemy Models:** These classes map our Python objects directly to the tables in our database, allowing us to work with Python code instead of raw SQL.
-   **Session Management:** This is the standard FastAPI pattern for handling database connections. The `get_db` function is a dependency that ensures each API request gets a database session and that the session is properly closed afterward.

In [4]:
db_code_prompt = f"""
You are a Python expert specializing in FastAPI and SQLAlchemy.

Based on the provided SQL schema, generate the necessary Python code to connect a FastAPI application to a SQLite database named 'onboarding.db'.

**SQL Schema Context:**
```sql
{sql_schema}
```

Please provide two separate, well-commented code blocks:

1.  **SQLAlchemy Models:** Create the Python classes that map to the `users` and `onboarding_tasks` tables.
2.  **Database Session Management:** Provide the standard boilerplate code for creating the SQLAlchemy engine, the `SessionLocal` class, and the `get_db` dependency for FastAPI.

Only output the raw Python code.
"""

print("--- Generating SQLAlchemy Models and Session Code ---")
if sql_schema:
    generated_db_code = get_completion(db_code_prompt, client, model_name, api_provider)
    print("\n--- Generated Database Code ---")
    print(generated_db_code)
else:
    print("Skipping DB code generation because schema is missing.")

--- Generating SQLAlchemy Models and Session Code ---

--- Generated Database Code ---
### SQLAlchemy Models
```python
from sqlalchemy import Column, Integer, String, Date, Boolean, ForeignKey, Enum, Text, TIMESTAMP, CheckConstraint
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, nullable=False)
    email = Column(String, nullable=False, unique=True)
    role = Column(String, nullable=False)
    department = Column(String)
    location = Column(String)
    hire_date = Column(Date)
    manager_id = Column(Integer, ForeignKey('users.id'))
    created_at = Column(TIMESTAMP, default=lambda: None)
    updated_at = Column(TIMESTAMP, default=lambda: None, onupdate=lambda: None)

    manager = relationship('User', remote_side=[id], backref='team_members')
    onboarding_tasks = rel

### Challenge 3 (Advanced): Integrating Live Database Logic

**Explanation:**
This final code represents the crucial role of the developer in an AI-assisted workflow. The AI provided the components (Pydantic models, SQLAlchemy models, endpoint structure), but the developer is responsible for the final integration, ensuring all the pieces work together seamlessly. This involves combining the generated code blocks and replacing the in-memory list operations with live SQLAlchemy database calls (`db.add`, `db.query`, `db.commit`, etc.).

In [5]:
# This block contains the complete, final code for app/main.py after manual integration.
final_api_code = """from fastapi import FastAPI, Depends, HTTPException
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Text, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session, relationship
from pydantic import BaseModel
from typing import List, Optional

# --- SQLAlchemy Setup ---
SQLALCHEMY_DATABASE_URL = "sqlite:///./artifacts/onboarding.db"
engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# --- SQLAlchemy Models ---
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)
    email = Column(String, unique=True, index=True)
    role = Column(String)
    tasks = relationship("OnboardingTask", back_populates="owner")

class OnboardingTask(Base):
    __tablename__ = 'onboarding_tasks'
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String, index=True)
    description = Column(Text)
    due_date = Column(Date)
    status = Column(String, default='Pending')
    user_id = Column(Integer, ForeignKey('users.id'))
    owner = relationship("User", back_populates="tasks")

# Create the database tables
Base.metadata.create_all(bind=engine)

# --- Pydantic Models ---
class UserBase(BaseModel):
    email: str
    name: str
    role: str

class UserCreate(UserBase):
    pass

class UserSchema(UserBase):
    id: int
    class Config:
        orm_mode = True

# --- FastAPI App ---
app = FastAPI()

# --- Dependency ---
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# --- API Endpoints ---
@app.post("/users/", response_model=UserSchema)
def create_user(user: UserCreate, db: Session = Depends(get_db)):
    db_user = db.query(User).filter(User.email == user.email).first()
    if db_user:
        raise HTTPException(status_code=400, detail="Email already registered")
    db_user = User(**user.dict())
    db.add(db_user)
    db.commit()
    db.refresh(db_user)
    return db_user

@app.get("/users/", response_model=List[UserSchema])
def read_users(skip: int = 0, limit: int = 100, db: Session = Depends(get_db)):
    users = db.query(User).offset(skip).limit(limit).all()
    return users

@app.get("/users/{user_id}", response_model=UserSchema)
def read_user(user_id: int, db: Session = Depends(get_db)):
    db_user = db.query(User).filter(User.id == user_id).first()
    if db_user is None:
        raise HTTPException(status_code=404, detail="User not found")
    return db_user
"""

print("--- Final Integrated API Code for app/main.py ---")
print(final_api_code)
save_artifact(final_api_code, "app/main.py")

--- Final Integrated API Code for app/main.py ---
from fastapi import FastAPI, Depends, HTTPException
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Text, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session, relationship
from pydantic import BaseModel
from typing import List, Optional

# --- SQLAlchemy Setup ---
SQLALCHEMY_DATABASE_URL = "sqlite:///./artifacts/onboarding.db"
engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# --- SQLAlchemy Models ---
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)
    email = Column(String, unique=True, index=True)
    role = Column(String)
    tasks = relationship("OnboardingTask", back_populates="owner")

class OnboardingTask(Base):
 

## Lab Conclusion

Congratulations! You have successfully generated and assembled a complete, database-connected backend API. You used an LLM to generate the boilerplate for both the API endpoints and the database models, and then performed the crucial engineering task of integrating them. You now have a working `main.py` file in your `app` directory that can create, read, update, and delete data in a live database. In the next lab, we will write a comprehensive test suite for this API.

> **Key Takeaway:** AI excels at generating boilerplate code (like models and endpoint structures), but the developer's critical role is in the final integration and wiring of these components into a coherent, working system.